In [1]:
'''
    20251124:
        把一些神经元mask掉，再看看generation


'''

'\n    20251124:\n        把一些神经元mask掉，再看看generation\n\n\n'

In [8]:
import transformers


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer


In [16]:
import copy

# function

In [26]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='auto'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

def set_neuron_zero(model, neuron_list):
    '''
    将指定的neuron设置为zero

    '''
    state_dict = model.state_dict()

    for ikey,_ in neuron_list:
        w_name = ikey.split('_index_')[0]
        neuron_index = int(ikey.split('_index_')[1])

        #set zero
        state_dict[w_name][neuron_index,:] = 0
    return model
    


def generate(model, tokenizer):
    

    textlist =[
        '中国',
        '人们总是做一件',
        "你是",
        "通义千问",
        "openAI"
    
    ]
    #textlist=["The garden blooms with vibrant colors, a gentle breeze carries the scent of roses, and the sun casts a warm glow."] # maxlen 64
    #textlist=['智慧殿堂中，书籍如海，知识无尽，探索永恒。艾拉追寻历史，灵感无限']# maxlen 64
    #textlist=['你是']# maxlen 64
    
    #textlist=['The garden blooms with vibrant colors, a gentle breeze carries the scent of roses, and the sun casts a warm glow.']
    textlist=['China is']
    bar = tqdm(total=len(textlist))
    
    
    device='cuda' # cpu cuda
    for itext in textlist:
        print('*'*20)
        input_text = itext
        print('input:', input_text)
        input_ids = tokenizer(input_text, return_tensors="pt").to(device)
        print('input_ids:', input_ids)
    
        outputs = model.generate(**input_ids, do_sample=False, num_beams=1, max_new_tokens=32) #max_length 
        print('outputs:', outputs)
        print('output:',tokenizer.batch_decode(outputs, skip_special_tokens=False))
        bar.update(1)

        
    




    

In [20]:
model, tokenizer = load_model('/root/autodl-fs/model_zoo/google/gemma-3-1b-it')

checkpoint: /root/autodl-fs/model_zoo/google/gemma-3-1b-it


In [21]:
import torch

In [22]:
neuron_en = torch.load('/root/autodl-fs/LAPE_res/202251124_en.pt', weights_only=False)
neuron_vi = torch.load('/root/autodl-fs/LAPE_res/202251124_vi.pt', weights_only=False)
neuron_zh = torch.load('/root/autodl-fs/LAPE_res/202251124_zh.pt', weights_only=False)

In [23]:
model_en = copy.deepcopy(model)
model_en = set_neuron_zero(model_en, neuron_en)

In [27]:
generate(model, tokenizer)

  0%|          | 0/1 [00:00<?, ?it/s]

********************
input: China is
input_ids: {'input_ids': tensor([[    2, 31939,   563]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}


100%|██████████| 1/1 [00:00<00:00,  1.29it/s]

outputs: tensor([[     2,  31939,    563,    496,   3262,   4155,    528,    506,   4185,
           8122, 236764,    532,   1061,   5527,   3877,    815,   1010,    496,
           3629,   5415,    528,    506,   1902, 236789, 236751,   5527,   2913,
         236761,   3153, 236764,   5818, 236789, 236751,   5527,   2028]],
       device='cuda:0')
output: ["<bos>China is a major player in the global economy, and its economic growth has been a significant factor in the world's economic development. However, China's economic model"]


In [28]:
generate(model_en, tokenizer)

  0%|          | 0/1 [00:00<?, ?it/s]

********************
input: China is
input_ids: {'input_ids': tensor([[    2, 31939,   563]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}


100%|██████████| 1/1 [00:00<00:00,  1.34it/s]

outputs: tensor([[     2,  31939,    563,    496, 151104,    529,  67885,    507,  13390,
         236764,    496,   1707,    529,    506, 236772, 236754,  29426,    522,
         236772,  74131, 236772,   7705, 236772,   1627, 236772,   9849, 236772,
         236746, 236772,   3397, 236772,    624, 236772,   1437, 236772]],
       device='cuda:0')
output: ['<bos>China is a whirlwind of opaquees industries, a fact of the-harnessing-together-that-ever-was-a-true-and-the-']


In [9]:
neuron_en

[('model.layers.0.mlp.up_proj.weight_index_381',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.0.mlp.up_proj.weight_index_470',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.0.mlp.up_proj.weight_index_2267',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.0.mlp.up_proj.weight_index_4680',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.2.mlp.up_proj.weight_index_342',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.gate_proj.weight_index_2771',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_1485',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_2480',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_3620',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.3.mlp.up_proj.weight_index_6512',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.4.mlp.gate_proj.weight_index_893',
  np.float64(-9.999994999180668e-07)),
 ('model.layers.4.mlp